In [1]:
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
def count_rows_in_csv_files(folder_path):
    file_counts = {}
    total_rows = 0
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            try:
                df = pd.read_csv(file_path, header=None)
                row_count = len(df) - 1
                file_counts[file_name] = row_count
                print(f"{file_name}: {row_count} rows (excluding header)")
                total_rows += row_count
            except Exception as e:
                print(f"Error reading {file_name}: {str(e)}")
    print(f"Total rows across all files: {total_rows}")
    return file_counts

folder_path = 'C:/Users/91912/Downloads/OCD_Files'
count_rows = count_rows_in_csv_files(folder_path)

    


combined_df.csv: 21842254 rows (excluding header)
df_hw.csv: 50700 rows (excluding header)
OCDetect_01_recording_05_03d0f5d8-eeee-40d4-ad75-5fefb70aa4f7.csv: 3332981 rows (excluding header)
OCDetect_01_recording_06_839c3df9-a71d-4520-9973-9c5344740611.csv: 2692263 rows (excluding header)
OCDetect_01_recording_07_2bd04f5f-7069-4f92-964e-59075a36d8fe.csv: 2332594 rows (excluding header)
OCDetect_01_recording_08_4631fb24-4762-4da6-80a9-7ad72624230d.csv: 2475882 rows (excluding header)
OCDetect_01_recording_09_851bcb34-b2ba-44bb-8d09-8e5316110569.csv: 7538 rows (excluding header)
OCDetect_01_recording_10_5b1f6bba-485e-4c1c-8af5-e80000767c78.csv: 7490 rows (excluding header)
OCDetect_01_recording_11_5cca782c-5706-4963-9682-fa9047e4a6a7.csv: 6476 rows (excluding header)
OCDetect_01_recording_12_dc492416-b349-49a2-8454-9b5c30c8a49c.csv: 4261 rows (excluding header)
OCDetect_01_recording_13_632d7391-7cb5-4b4f-ba4f-8bc1f62ce55c.csv: 2566 rows (excluding header)
OCDetect_01_recording_14_2ec33f65

In [3]:
def combine_csv_files(folder_path):
    combined_data = pd.DataFrame()
    file_list = os.listdir(folder_path)
    data_frames = []

    for i, file_name in enumerate(file_list):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            if i == 0:
                data = pd.read_csv(file_path, header=None)
            else:
                data = pd.read_csv(file_path, header=None,skiprows = 1)
            data_frames.append(data)

    combined_data = pd.concat(data_frames, ignore_index=True)
    return combined_data

In [4]:
combined_df = combine_csv_files(folder_path)

MemoryError: Unable to allocate 89.0 MiB for an array with shape (5, 2332594) and data type float64

In [ ]:
#combined_df.to_hdf(f'{folder_path}/combined_df.h5', key='df', mode='w')
combined_df.to_csv(f'{folder_path}/combined_df.csv', index=False)

In [ ]:
combined_df.head()

In [ ]:
print(len(combined_df))

In [ ]:
dataframe = combined_df

In [ ]:
print(len(dataframe))

In [ ]:
dataframe.columns = dataframe.iloc[0]
dataframe = dataframe[1:]
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
dataframe.head()

In [ ]:
ignore_unique = pd.to_numeric(dataframe['ignore'], errors='coerce').dropna().astype(int).value_counts().to_dict()
print(ignore_unique)

In [ ]:
dataframe_numeric = dataframe.apply(pd.to_numeric, errors='coerce')
new_dataframe = dataframe_numeric[dataframe_numeric['ignore'].eq(0)]
print(len(new_dataframe))

In [ ]:
df = new_dataframe[['acc x','acc y','acc z','gyro x','gyro y','gyro z', 'relabeled']]
y = df['relabeled']


In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
unique_values = df['relabeled'].unique()
print(unique_values)

In [ ]:
df_hw = df[df['relabeled'] != 0]
unique_values = df_hw['relabeled'].unique()
print(unique_values)
print(len(df_hw))

In [ ]:
df_hw.head()

In [ ]:
df_hw.to_csv(f'{folder_path}/df_hw.csv', index=False)

In [ ]:
y = df_hw['relabeled']
df_hw.drop(['relabeled'], axis=1, inplace=True)

In [ ]:
df_hw.head()

In [ ]:
df_hw.shape

In [ ]:
X = []
labels = []
window_size = 150
step_size = 150

In [ ]:
y

In [ ]:
for i in range(0, df_hw.shape[0]-window_size+1, step_size):
  frame = df_hw.iloc[i:i+window_size]
  label = Counter(y.iloc[i:i+window_size]).most_common(1)[0][0]
  X.append(frame)
  labels.append(label)

In [ ]:
X_arr = np.array(X)
X_arr.shape

In [ ]:
label_arr = np.array(labels)
label_arr.shape

In [ ]:
np.unique(label_arr)

In [ ]:
X_flattened = X_arr.reshape(X_arr.shape[0], X_arr.shape[1]*X_arr.shape[2])
X_flattened.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_flattened, label_arr, test_size=0.25, random_state=42, shuffle=True)

In [ ]:
#X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)

In [ ]:
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
y_train.shape


In [ ]:
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)
y_pred_test = classifier.predict(X_test)

In [ ]:
print("F1: ", metrics.f1_score(y_test, y_pred_test, average='weighted'))
print("Precision: ", metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred_test, average='weighted'))
print()
print('Confusion Matrix')
print(metrics.confusion_matrix(y_test, y_pred_test))